# 중간제출물 코드정리

In [47]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import datetime 

## 1. 데이터셋 준비

- 데이터셋 불러오기

In [2]:
# 액티브 중소법인 재무보유 sheet 별로 불러오기
df_active_summary_out = pd.read_excel('../dataset/액티브 중소법인 재무보유.xlsx', sheet_name='기업개요_외감')
df_active_summary = pd.read_excel('../dataset/액티브 중소법인 재무보유.xlsx', sheet_name='기업개요')
df_active_inactive = pd.read_excel('../dataset/액티브 중소법인 재무보유.xlsx', sheet_name='휴폐업이력')

# 휴폐업 중소법인 재무보유 sheet 별로 불러오기
df_inactive_summary = pd.read_excel('../dataset/휴폐업 중소법인 재무보유.xlsx', sheet_name='기업개요')
df_inactive_inactive = pd.read_excel('../dataset/휴폐업 중소법인 재무보유.xlsx', sheet_name='휴폐업이력')

- 컬럼명 한글컬럼명으로 바꾸기

In [3]:
# 개방데이터 레이아웃에서 한글칼럼명 가져오기

df_columns = pd.read_excel('../dataset/넘블 챌린지 개방데이터 레이아웃.xlsx', sheet_name='최종')
df_columns = df_columns[['영문칼럼명', '한글칼럼명']]
dict_columns = dict(zip(df_columns['영문칼럼명'], df_columns['한글칼럼명']))

In [4]:
# 액티브 중소법인 재무보유 영문칼럼명에서 한글칼럼명으로 바꾸기
df_active_summary_out.rename(columns = dict_columns, inplace=True)
df_active_summary.rename(columns = dict_columns, inplace=True)
df_active_inactive.rename(columns = dict_columns, inplace=True)

# 휴폐업 중소법인 재무보유 영문칼럼명에서 한글칼럼명으로 바꾸기
df_inactive_summary.rename(columns = dict_columns, inplace=True)
df_inactive_inactive.rename(columns = dict_columns, inplace=True)

- 중소법인 기업개요 모두 합치기

In [5]:
# 액티브 중소법인 재무보유 기업개요 합치기
# 휴폐업 중소법인 기업개요에서 외감 구분 컬럼이 없기 떄문에 활용가능성?

df_active_summary = pd.concat([df_active_summary_out, df_active_summary])
df_active_summary['휴폐업 유무'] = 'N'
df_inactive_summary['휴폐업 유무'] = 'Y'
df_summary = pd.concat([df_active_summary, df_inactive_summary])

In [35]:
print(len(df_active_summary_out), len(df_active_summary), len(df_inactive_summary))

28982 227633 6739


In [6]:
df_summary.to_csv('../dataset/merge.csv', index=False)

- 산업코드 대분류 불러오기

In [7]:
# 중소법인 기업개요
non_financial = pd.read_csv('../dataset/merge.csv')

# 산업코드1에서 결측치 드랍
non_financial = non_financial.dropna(subset= ['산업코드1'])

/var/folders/8t/p9vrbnhn7xj3s_2zmct430yw0000gn/T/ipykernel_22561/3468461061.py:2: DtypeWarning: Columns (2,28) have mixed types. Specify dtype option on import or set low_memory=False.
  non_financial = pd.read_csv('../dataset/merge.csv')


In [8]:
# 산업코드 앞 3자리 추출(산업코드가 4자리인 레코드 5자리로 바꾸기)

def to_three(s):
    if len(str(int(s))) == 4:
        tmp = '0'+str(int(s))    
    else:
        tmp = str(int(s))
        
    return tmp[:3]

In [9]:
non_financial['산업코드1'] = non_financial['산업코드1'].apply(lambda x: to_three(x))

In [10]:
# 표준산업코드
standard_ind_code = pd.read_excel('../dataset/산업분류/standard_ind_code.xlsx', header=2)
standard_ind_code = standard_ind_code.fillna(method='ffill')

In [11]:
# 산업코드 앞 3자리 통일(산업코드가 2자리인 레코드 3자리로 바꾸기)

def to_three2(s):
    if len(str(int(s))) == 2:
        tmp = '0'+str(int(s))    
    else:
        tmp = str(int(s))
        
    return tmp

In [12]:
standard_ind_code['코드'] = standard_ind_code['코드'].apply(lambda x: to_three2(x))

In [13]:
new_non_financial = pd.merge(non_financial, standard_ind_code, left_on= '산업코드1', right_on='코드')

In [14]:
new_non_financial = new_non_financial.drop(columns='산업코드1')
new_non_financial= new_non_financial.rename(columns={'항목명': '대분류', '코드': '분류코드', '항목명.1': '세부항목'})

In [15]:
new_non_financial

,사업자등록번호,기업접두명,CMP_NM1,기업접미명,기업영문명,업종,기업규모,공기업구분 1.공기업 2.일반기업,개인법인구분 1.법인 2.개인,본점지점구분 1.본점 2.지점,...,홈페이지URL,대표자명,직원수,기업명,CMP_TYP,TAXN_TYP,휴폐업 유무,대분류,분류코드,세부항목
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,...,www.hanilgas.com,홍순철외 1명,43.0,NaN,NaN,NaN,N,도매 및 소매업(45~47),477,연료 소매업
1,6068158196,(주),오에스에너지,NaN,"Oil Server Energy Co.,Ltd.",M,2,2,1,1,...,NaN,강성용,6.0,NaN,NaN,NaN,N,도매 및 소매업(45~47),477,연료 소매업
2,1118100750,NaN,신흥에너지산업,(주),"SHINHEUNG ENERGY CO.,LTD.",M,2,2,1,1,...,NaN,최덕식,18.0,NaN,NaN,NaN,N,도매 및 소매업(45~47),477,연료 소매업
3,6098103778,NaN,광신개발,(주),"KWANGSHIN DEVELOMENT CO.,LTD.",M,2,2,1,1,...,www.gs.co.kr,최우찬,10.0,NaN,NaN,NaN,N,도매 및 소매업(45~47),477,연료 소매업
4,2178102639,(주),노일가스,NaN,"NOILGAS CO.,LTD.",M,2,2,1,1,...,NaN,최계락,18.0,NaN,NaN,NaN,N,도매 및 소매업(45~47),477,연료 소매업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231214,1318400774,NaN,겐트대학교글로벌캠퍼스,NaN,Genteu University Global Campus,M,2,2,1,1,...,www.ghent.ac.kr,안느드페페,51.0,NaN,4.0,2.0,N,국제 및 외국기관(99),990,국제 및 외국기관
231215,6168204511,NaN,남녕학원,NaN,Namnyeong Institute,M,2,2,1,1,...,NaN,백동홍,0.0,NaN,4.0,3.0,N,교육 서비스업(85),852,중등 교육기관
231216,4098611727,NaN,에스에이치텍,(주),"SH Tech Co.,Ltd.",M,2,2,1,1,...,NaN,소재목,5.0,NaN,4.0,3.0,N,보건업 및 사회복지 서비스업(86~87),862,의원
231217,4738600600,(주),싸이토딕스,NaN,"Cy To Dix Co., Ltd.",M,2,2,1,1,...,NaN,선우요섭,6.0,NaN,4.0,3.0,N,보건업 및 사회복지 서비스업(86~87),862,의원


In [16]:
new_non_financial.to_csv('../dataset/non_financial.csv', index=False)

## 2. 피처엔지니어링

In [57]:
non_financial = pd.read_csv('../dataset/non_financial.csv')

- 분석에 필요없는 컬럼 제거하기

In [58]:
non_financial.columns

Index(['사업자등록번호', '기업접두명', 'CMP_NM1', '기업접미명', '기업영문명', '업종', '기업규모',
       '공기업구분 1.공기업 2.일반기업', '개인법인구분 1.법인 2.개인', '본점지점구분 1.본점 2.지점',
       '국외투자법인여부', '벤처기업여부', '상장코드', '산업코드차수', '산업코드2', '산업코드3', '공공기관유형',
       '중견기업보호여부', '본점기업코드', '설립일자', '설립구분', '상장일자', '주요사업내용', '국가명',
       '홈페이지URL', '대표자명', '직원수', '기업명', 'CMP_TYP', 'TAXN_TYP', '휴폐업 유무', '대분류',
       '분류코드', '세부항목'],
      dtype='object')

In [59]:
drop_list_1 = ['기업명', '기업영문명', '홈페이지URL', 
               '대표자명', '기업접두명', '기업접미명', '주요사업내용',
               '업종', '산업코드2', '산업코드3', '분류코드', '세부항목'
               ] # no need
drop_list_2 = ['기업규모', '개인법인구분 1.법인 2.개인', '산업코드차수'] # CONSTANT
drop_list_3 = ['CMP_NM1', 'CMP_TYP', 'TAXN_TYP'] # only columns in df_active
drop_list = drop_list_1 + drop_list_2 + drop_list_3

# 필요없는 컬럼 드랍하기
non_financial = non_financial[[c for c in non_financial.columns if c not in drop_list]]
non_financial.columns

Index(['사업자등록번호', '공기업구분 1.공기업 2.일반기업', '본점지점구분 1.본점 2.지점', '국외투자법인여부',
       '벤처기업여부', '상장코드', '공공기관유형', '중견기업보호여부', '본점기업코드', '설립일자', '설립구분',
       '상장일자', '국가명', '직원수', '휴폐업 유무', '대분류'],
      dtype='object')

In [60]:
non_financial= non_financial.rename(columns={'공기업구분 1.공기업 2.일반기업':'공기업구분', 
                                             '본점지점구분 1.본점 2.지점': '본점지점구분',
                                             '휴폐업 유뮤': '휴폐업유무'})

In [61]:
# 상장코드

non_financial[non_financial['상장코드'].notnull()]
non_financial['상장코드'] = non_financial['상장코드'].notnull().replace({True: 'Y', False: 'N'})
non_financial = non_financial.rename(columns={'상장코드': '상장유무'})
non_financial

,사업자등록번호,공기업구분,본점지점구분,국외투자법인여부,벤처기업여부,상장유무,공공기관유형,중견기업보호여부,본점기업코드,설립일자,설립구분,상장일자,국가명,직원수,휴폐업 유무,대분류
0,1138111739,2,1,N,N,N,0,N,NaN,19660419.0,1.0,NaN,NaN,43.0,N,도매 및 소매업(45~47)
1,6068158196,2,1,N,N,N,0,N,NaN,19891001.0,1.0,NaN,NaN,6.0,N,도매 및 소매업(45~47)
2,1118100750,2,1,N,N,N,0,N,NaN,19740731.0,1.0,NaN,NaN,18.0,N,도매 및 소매업(45~47)
3,6098103778,2,1,N,N,N,0,N,NaN,19810301.0,1.0,NaN,NaN,10.0,N,도매 및 소매업(45~47)
4,2178102639,2,1,N,N,N,0,N,NaN,19881229.0,1.0,NaN,NaN,18.0,N,도매 및 소매업(45~47)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231214,1318400774,2,1,N,N,N,0,N,NaN,20130620.0,99.0,NaN,NaN,51.0,N,국제 및 외국기관(99)
231215,6168204511,2,1,N,N,N,0,N,NaN,NaN,8.0,NaN,NaN,0.0,N,교육 서비스업(85)
231216,4098611727,2,1,N,Y,N,0,N,NaN,20091112.0,1.0,NaN,NaN,5.0,N,보건업 및 사회복지 서비스업(86~87)
231217,4738600600,2,1,N,Y,N,0,N,NaN,20170124.0,1.0,NaN,NaN,6.0,N,보건업 및 사회복지 서비스업(86~87)


In [69]:
# 설립일자, 상장일자 

non_financial['설립일자'] = pd.to_datetime(non_financial['설립일자'].apply(lambda x: str(int(x))), format='%Y%m%d')
non_financial['상장일자'] = pd.to_datetime(non_financial['상장일자'].apply(lambda x: str(int(x))), format='%Y%m%d')
non_financial

ValueError: cannot convert float NaN to integer